<a href="https://colab.research.google.com/github/dldbwls00/NLP-sentiment-analysis-project/blob/main/kobert_sentiment_0109ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kobert finetuning

In [1]:
# package install
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install tokenizers
!pip install torch
!pip install pandas
!pip install transformers==4.30.0
!pip install gluonnlp

In [2]:
# kobert install
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-evw3tyhv/kobert-tokenizer_cc8fe199b951491c8102dbcfc211393e
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-evw3tyhv/kobert-tokenizer_cc8fe199b951491c8102dbcfc211393e
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [4]:
# importing kobert using hugging face
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
# gpu setting
#device = torch.device("cuda:0")
#!nvidia-smi
device = torch.device("cpu")

In [6]:
# tokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [7]:
# connecting google drive - colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# loading dataset
import pandas as pd
conv_training = pd.read_excel('/content/drive/MyDrive/dataset/conv_training.xlsx')
#conv_validation = pd.read_excel('/content/drive/MyDrive/dataset/conv_validation.xlsx')

In [9]:
# dataset check - training
print(len(conv_training))
conv_training.sample(n=5)

51630


,Unnamed: 0,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
47511,47512,청년,여성,"진로,취업,직장",해당없음,당황,혼란스러운,요즘 친구들이랑 만나면 자기 연봉이 얼마인지 이 얘기만 해서 스트레스야.,친구들이 연봉 이야기만 해서 스트레스를 받으시는군요.,연봉 얘기를 들을 때마다 열심히 하지 않아서 좋은 회사에 못 들어간 게 너무 후회돼.,지금 감정에서 벗어나기 위해 무엇을 할 수 있을까요?,당분간은 친구들을 안 만나는 것이 좋겠어.,친구들과 거리를 두는 것으로 기분이 나아지길 바라요.
35033,35034,청년,남성,"연애,결혼,출산",해당없음,슬픔,눈물이 나는,아내가 교통사고를 당해 크게 다쳤다는 전화를 받았어. 걱정돼서 눈물이 나.,아내가 교통사고를 당해 크게 다치셨군요. 걱정돼서 슬프시겠어요.,입원해야 할 거 같은데 간호를 위해 휴가를 낼까 고민 중이야.,휴가를 내려고 고민 중이시군요. 어떻게 하는 것이 가장 좋을까요?,역시 입원을 해야 하니 휴가를 내는 것이 가장 좋을 거 같아.,휴가를 내는 것 외에 아내분을 위해 또 무엇을 할 수 있을까요?
37808,37809,청소년,여성,학업 및 진로,해당없음,분노,툴툴대는,하고 싶은 게 없는데 계속 진로를 결정하래.,걱정이 많으시겠어요.,무작정 스터디하는 것도 힘든데 진로는 또 왜 이렇게 빨리 선택해야 하는 걸까?,당신의 밝은 앞날이 기다리고 있을 거예요.,NaN,NaN
24677,24678,노년,남성,재정,만성질환 무,분노,방어적인,계속 돈 좀 빌려달라고 찾아오는 자식을 보니 화가 많이 나.,자녀 분 문제로 화가 많이 나셨군요.,자식이 돈에 대한 심각성을 너무 몰라서 호되게 한번 당해봤으면 좋겠어.,자녀분들이 조금 더 지혜로워지시기를 원하시는 군요. 이 문제를 해결하기 위한 방법이...,더 이상 내가 도움 주지 않을 거라는 걸 확실히 해둬야겠어.,자녀들과의 대화가 잘 풀렸으면 좋겠어요.
37711,37712,청소년,여성,학교폭력/따돌림,해당없음,상처,괴로워하는,요즘 학교에서 괴롭힘을 당해서 매우 괴로워.,친구들에게 폭력에 괴롭힘 당하시는 건가요?,최근 갑자기 폭력을 당하고 있어서 너무 괴로운 마음이야.,어떻게 하면 처리할 수 있을까요?,선생님께 말을 해보는게 좋겠지?,잘 해결되시길 성원할게요.


In [10]:
# dataset check - validation
#print(len(conv_validation))
#conv_validation.sample(n=5)

In [11]:
# converting label
conv_training.loc[(conv_training['감정_대분류'] == '기쁨'), '감정_대분류'] = 0
conv_training.loc[(conv_training['감정_대분류'] == '당황'), '감정_대분류'] = 1
conv_training.loc[(conv_training['감정_대분류'] == '분노'), '감정_대분류'] = 2
conv_training.loc[(conv_training['감정_대분류'] == '불안'), '감정_대분류'] = 3
conv_training.loc[(conv_training['감정_대분류'] == '상처'), '감정_대분류'] = 4
conv_training.loc[(conv_training['감정_대분류'] == '슬픔'), '감정_대분류'] = 5

In [12]:
# drop NaN + merge
conv_training['사람문장1'] = conv_training['사람문장1'].replace(pd.np.nan, '')
conv_training['사람문장2'] = conv_training['사람문장2'].replace(pd.np.nan, '')
conv_training['사람문장3'] = conv_training['사람문장3'].replace(pd.np.nan, '')
conv_training['sentences'] = conv_training['사람문장1'] + " " + conv_training['사람문장2'] + " " + conv_training['사람문장3']

<ipython-input-12-3ccb88233337>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  conv_training['사람문장1'] = conv_training['사람문장1'].replace(pd.np.nan, '')
<ipython-input-12-3ccb88233337>:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  conv_training['사람문장2'] = conv_training['사람문장2'].replace(pd.np.nan, '')
<ipython-input-12-3ccb88233337>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  conv_training['사람문장3'] = conv_training['사람문장3'].replace(pd.np.nan, '')


In [13]:
# dataset check - preprocessing conv_training
extracted = pd.DataFrame({
        'emotion': conv_training['감정_대분류'],
        'sentences': conv_training['sentences']
        })

print(len(extracted))
extracted.sample(n=5)

51630


,emotion,sentences
20696,3,정말 좋아하는 친구이지만 돈이 얽혀있을 때는 사기를 당할까봐 걱정돼. 그 친구는 돈...
49971,1,취업이 안 되는 게 내가 준비를 열심히 하지 않은 이유 때문인 것 같아 죄책감이 들...
48556,1,아내가 내 의처증 때문에 힘들다고 이혼을 요구해. 정말 미안하고 죄책감이 들어. 아...
35848,5,학교폭행에서 벗어나고 싶은데 방도가 없어. 아무런 생각이 안 나고 너무 괴로워. 내...
49096,4,사랑만 있으면 행복할 것 같은 결혼이었는데 가난에 시달리니 불행해져. 초라하게 변해...


In [14]:
# export file
#extracted.to_excel('extracted.xlsx')

In [15]:
# extract column -> to array
data_list = []
for q, label in zip(extracted['sentences'], extracted['emotion']) :
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [16]:
# array check
print(len(data_list))
print(data_list[0])
print(data_list[10000])
print(data_list[20000])
print(data_list[30000])
print(data_list[40000])
print(data_list[50000])
print(data_list[-1])

51630
['일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고. ', '2']
['나는 늘 모두에게 사랑받아야 한다고 생각하면서 살아왔어. 오늘 아침에 엄마랑 싸웠는데 엄마가 더 나를 사랑하지 않을까 봐 무서워.  아니 그러지는 않았어. ', '3']
['오늘 갑자기 사장님이 직장을 그만두라고 하셨어. 회사 상황이 좋지 못해서 한 명씩 사직시키고 있다는데 배신당한 기분이야. 부당해고로 신고할까 해.', '4']
['이번에 주식을 샀는데 주가가 너무 떨어져 손해가 이만저만이 아냐. 지금은 없어. 난 왜 이렇게 되는일이 없나 몰라. ', '5']
['다리 절뚝거린다고 내가 이상한 사람도 아닌데 말이야. 친구들도 멀리하니 슬프네! 젊은 시절 일하다가 다리를 다쳤는데 완전히 치료하지 못했어. 친구들도 자식들도 나를 창피해하는 거 같아. 억울하지 내가 피해를 준 것도 없는데 말이야. 주변 사람들 때문에 더 외롭고 힘들어.', '5']
['남편이 몸이 아파서 입원을 했는데 나도 몸이 성치 않은데 병간호를 해야 해서 힘들어. 슬프네. 돈이라도 많으면 간병인을 쓰겠는데 이것도 여의치 않고 이러다 나도 병나겠어. 나라도 몸이 아프지 않게 잘 먹고 몸 관리를 잘해야겠어.', '5']
['남편과 결혼한 지 사십 년이야. 이제 사람 만나는 것도 버겁고 알던 사람도 점점 사라져. 사람들을 만나는 것이 어려워. 자꾸 사람들을 의심하게만 되고 말이야. 사람들을 볼 때 의심하고 불신하는 마음을 억눌러야겠어. 사람들을 색안경을 끼고 보지 않게.', '4']


In [17]:
# splitting data
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

print(len(dataset_train))
print(len(dataset_test))

38722
12908


In [18]:
"""class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
"""


class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [19]:
# setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [20]:
# preprocessing
tok = tokenizer.tokenize
#tokenizer = get_tokenizer()
#tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [21]:
# kobert training model
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [22]:
# load model
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [23]:
# prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# model training
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-24-4b508a570d4d>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/606 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 1.8940086364746094 train acc 0.140625
